In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import matplotlib.pyplot as plt
#from lightgbm import LGBMRegressor
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
#importing the dataset
import pandas as pd
df = pd.read_csv('/home/jehona/PythonDataScienceHandbook-master/renfe.csv',sep=',')
df.head(10)

,Unnamed: 0,insert_date,origin,destination,start_date,end_date,train_type,price,train_class,fare
0,0,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 06:20:00,2019-05-29 09:16:00,AV City,38.55,Turista,Promo
1,1,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 07:00:00,2019-05-29 09:32:00,AVE,53.40,Turista,Promo
2,2,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 07:30:00,2019-05-29 09:51:00,AVE,47.30,Turista,Promo
3,3,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 08:00:00,2019-05-29 10:32:00,AVE,69.40,Preferente,Promo
4,4,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 08:30:00,2019-05-29 11:14:00,ALVIA,NaN,Turista,Promo
5,5,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 09:00:00,2019-05-29 11:38:00,AVE,60.30,Turista,Promo
6,6,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 09:45:00,2019-05-29 12:27:00,INTERCITY,62.20,Turista,Flexible
7,7,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 10:00:00,2019-05-29 12:32:00,AVE,47.30,Turista,Promo
8,8,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 11:00:00,2019-05-29 13:32:00,AVE,53.40,Turista,Promo
9,9,2019-04-19 05:31:43,MADRID,SEVILLA,2019-05-29 11:05:00,2019-05-29 13:41:00,ALVIA,NaN,Turista,Promo


In [3]:
df = df.dropna()

In [4]:
df.drop(["Unnamed: 0"], axis = 1, inplace = True)
df.drop(["insert_date"], axis = 1, inplace = True)

In [5]:
df['start_date'] =pd.to_datetime(df['start_date'])
df['end_date']=pd.to_datetime(df['end_date'])

In [6]:
# create new columns
import numpy as np
df['duration']=df['end_date']-df['start_date']
df['duration_mins']=df['duration']/np.timedelta64(1, 'm')
df['start_time']=df['start_date'].dt.time

In [7]:
# add a column with duration
import datetime as dt
df['duration (h)'] = (df['end_date']-df['start_date']).dt.seconds/3600


In [8]:
coordinates = {'MADRID':[40.4065,3.6896],\
               'SEVILLA':[37.3911,5.9755],\
               'PONFERRADA':[42.5454,6.6023],\
               'BARCELONA':[41.3790,2.1399],\
               'VALENCIA':[39.4666,0.3773]}

# create columns with origin and destination coordinates
df['origin_coordinates'] = df['origin'].apply(lambda x: coordinates[x])
df['destination_coordinates'] = df['destination'].apply(lambda x: coordinates[x])

# put together a function for calculating distance
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
    
    return d

# create the distance function
df['distance (km)'] = df.apply(lambda row: distance(row['origin_coordinates'],row['destination_coordinates']), axis=1)

# get basic statistics for the function
df['distance (km)'].describe()

# remove columns not needed aymore
df = df.drop(columns=['origin_coordinates','destination_coordinates'])

In [9]:
# create a column with origin and destination (this variable is effectively equivalent to the distance one)
df['origin - destination'] = df['origin']+' - '+df['destination']

In [10]:
# create variables to reflect seasonal price movements
df['month'] = df['start_date'].dt.month
df['weekday'] = df['start_date'].dt.weekday
df['day'] = df['start_date'].dt.day
df['hour'] = df['start_date'].dt.hour
df['minute'] = df['start_date'].dt.minute



In [11]:
df.head()

,origin,destination,start_date,end_date,train_type,price,train_class,fare,duration,duration_mins,start_time,duration (h),distance (km),origin - destination,month,weekday,day,hour,minute
0,MADRID,SEVILLA,2019-05-29 06:20:00,2019-05-29 09:16:00,AV City,38.55,Turista,Promo,02:56:00,176.0,06:20:00,2.933333,389.26503,MADRID - SEVILLA,5,2,29,6,20
1,MADRID,SEVILLA,2019-05-29 07:00:00,2019-05-29 09:32:00,AVE,53.40,Turista,Promo,02:32:00,152.0,07:00:00,2.533333,389.26503,MADRID - SEVILLA,5,2,29,7,0
2,MADRID,SEVILLA,2019-05-29 07:30:00,2019-05-29 09:51:00,AVE,47.30,Turista,Promo,02:21:00,141.0,07:30:00,2.350000,389.26503,MADRID - SEVILLA,5,2,29,7,30
3,MADRID,SEVILLA,2019-05-29 08:00:00,2019-05-29 10:32:00,AVE,69.40,Preferente,Promo,02:32:00,152.0,08:00:00,2.533333,389.26503,MADRID - SEVILLA,5,2,29,8,0
5,MADRID,SEVILLA,2019-05-29 09:00:00,2019-05-29 11:38:00,AVE,60.30,Turista,Promo,02:38:00,158.0,09:00:00,2.633333,389.26503,MADRID - SEVILLA,5,2,29,9,0


In [12]:
# features
features = pd.concat([pd.get_dummies(df[['train_type','train_class','fare','month','weekday','day','hour','minute','duration (h)']]),\
           pd.get_dummies(df[['distance (km)']].astype(str)),\
                     ],axis=1)

In [13]:
from sklearn.preprocessing import OneHotEncoder


onehotencoder = OneHotEncoder()
X = onehotencoder.fit_transform(features.values)

/home/jehona/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [17]:
y = df['price']

In [18]:
n_samples = len(df['price'])
indices = np.arange(n_samples)

In [19]:
# split into train and test 
X_train, X_test, y_train, y_test, idx1, idx2 = train_test_split(X, y, indices, test_size=0.1)



In [20]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [21]:
train_score_lr = model.score(X_train, y_train)
test_score_lr = model.score(X_test, y_test)
print("Train Score:", train_score_lr)
print("Test Score:", test_score_lr)
print()

Train Score: 0.8539507086135945
Test Score: 0.8528256266426281



In [22]:
#y_pred contains the predicted prices
y_pred = model.predict(X_test)

In [24]:
Price = df.iloc[:,[5]].as_matrix()

/home/jehona/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [25]:
for i in range(len(y_pred)):
    Price[idx2[i]] = y_pred[i]
    

In [26]:
df.drop(["price"], axis = 1, inplace = True)

In [27]:
df['price'] = Price

In [28]:
df.to_csv('Tickets.csv', index=False, sep=',')
#Tickets.csv is the file which contains the updated dataset, 
#where the price column contains the values predicted by the algorithm
